# Лабораторная работа №2
### Выполнил студент БПИ2301 Орлова Владислава

Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [12]:
import random

def random_generate(size):
    data = random.sample(range(size*10), size)
    return data
first_date = random_generate(50)
print(sorted(first_date))


[0, 2, 11, 12, 17, 18, 22, 23, 30, 43, 58, 72, 89, 96, 101, 102, 108, 115, 116, 119, 126, 132, 147, 164, 166, 176, 182, 233, 241, 247, 275, 276, 281, 288, 297, 299, 320, 353, 354, 378, 398, 402, 412, 416, 437, 443, 447, 477, 485, 492]


### Задание №1


### Бинарный поиск


In [43]:
import timeit

def find_num(sort_arr_nums, candidate):
    first = 0
    last = len(sort_arr_nums) - 1
    while (first <= last):
        middle = (first + last) // 2
        if candidate == sort_arr_nums[middle]:
            return candidate
        if candidate < sort_arr_nums[middle]:
            last = middle - 1
        else:
            first = middle + 1
    return -1
time = timeit.timeit(lambda: find_num(sorted(first_date), 24), number=1)
print(f"Время выполнения: {time} секунд")
print(find_num(sorted(first_date), 166))

Время выполнения: 1.7899961676448584e-05 секунд
166


### Бинарное дерево



In [53]:
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def insert_node(tree_root, new_value):
    if tree_root is None:
        return TreeNode(new_value)
    if new_value < tree_root.value:
        tree_root.left = insert_node(tree_root.left, new_value)
    else:
        tree_root.right = insert_node(tree_root.right, new_value)
    return tree_root

def find_value(tree_root, search_value):
    if tree_root is None or tree_root.value == search_value:
        return tree_root
    if search_value < tree_root.value:
        return find_value(tree_root.left, search_value)
    return find_value(tree_root.right, search_value)

tree_root = None
for number in first_date:
    tree_root = insert_node(tree_root, number)

time = timeit.timeit(lambda: find_value(tree_root, 166), number=1)
print(f"Время выполнения: {time} секунд")

search_result = find_value(tree_root, 166)
if search_result:
    print("Всё гуд")
else:
    print("Такого числа нет")



Время выполнения: 6.199989002197981e-06 секунд
Всё гуд


### Фибоначчиев


In [54]:
def search_fibonachchi(arr, target):
    val_left2 = 0
    val_left1 = 1
    val = val_left1 + val_left2

    while val < len(arr):
        val_left2 = val_left1
        val_left1 = val
        val = val_left1 + val_left2

    index = -1

    while val > 1:
        i = min(index + val_left2, len(arr) - 1)
        if arr[i] < target:
            val = val_left1
            val_left1 = val_left2
            val_left2 = val - val_left1
            index = i
        elif arr[i] > target:
            val = val_left2
            val_left1 = val_left1 - val_left2
            val_left2 = val - val_left1
        else:
            return i, arr[i]

    if val_left1 and index < len(arr) - 1 and arr[index + 1] == target:
        return index + 1, arr[index + 1]

    return -1

time = timeit.timeit(lambda:search_fibonachchi(sorted(first_date), 166), number=1)
print(f"Время выполнения: {time} секунд")

print(search_fibonachchi(sorted(first_date), 166))

Время выполнения: 1.8400023691356182e-05 секунд
(24, 166)


### Интерполяционный


In [55]:
def interpolation_search(arr, val):
    low = 0
    high = (len(arr) - 1)
    while low <= high and val >= arr[low] and val <= arr[high]:
        index = low + int(((float(high - low) / ( arr[high] - arr[low])) * ( val - arr[low])))
        if arr[index] == val:
            return index, arr[index]
        if arr[index] < val:
            low = index + 1
        else:
            high = index - 1
    return -1

time = timeit.timeit(lambda: interpolation_search(sorted(first_date), 166), number=1)
print(f"Время выполнения: {time} секунд")

print(interpolation_search(sorted(first_date), 166))

Время выполнения: 0.00010040000779554248 секунд
(24, 166)


### Задание №2


### Простое рехэширование

In [56]:
def hash_function(key, size):
    return key % size

def rehash_function(key, attempt, size):
    return (hash_function(key, size) + attempt) % size

def search_with_rehashing(arr, target):
    size = len(arr)
    attempt = 0
    index = hash_function(target, size)
    
    while attempt < size:
        if arr[index] == target:
            return arr[index], index
        attempt += 1
        index = rehash_function(target, attempt, size)
    
    return -1

time = timeit.timeit(lambda: search_with_rehashing(sorted(first_date), 166), number=1)
print(f"Время выполнения: {time} секунд")

result = search_with_rehashing(sorted(first_date), 166)
if result != -1:
    print(f"Число найдено под индексом {result}.")
else:
    print(f"Число не найдено.")

Время выполнения: 1.6600009985268116e-05 секунд
Число найдено под индексом (166, 24).


### Рехэширование с помощью псевдослучайных чисел


In [57]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        return key % self.size

    def rehash_function(self, key, attempt):
        random_seed = key + attempt
        pseudo_random = (random_seed * 48271) % 2147483647
        return (self.hash_function(key) + pseudo_random) % self.size

    def insert(self, key, value):
        attempt = 0
        index = self.hash_function(key)
        
        while self.table[index] is not None:
            if self.table[index][0] == key:
                self.table[index] = (key, value)
                return
            attempt += 1
            index = self.rehash_function(key, attempt)
        
        self.table[index] = (key, value)

    def search(self, key):
        attempt = 0
        index = self.hash_function(key)
        
        while self.table[index] is not None:
            if self.table[index][0] == key:
                return self.table[index][1]
            attempt += 1
            index = self.rehash_function(key, attempt)
        
        return -1
    
hash_table = HashTable(100)
for num in first_date:
    hash_table.insert(num, num)

time = timeit.timeit(lambda: hash_table.search(166), number=1)
print(f"Время выполнения: {time} секунд")

result = hash_table.search(166)
if result != -1:
    print(f"Число найдено: {result}")
else:
    print("Число не найдено")

Время выполнения: 4.399975296109915e-06 секунд
Число найдено: 166


### Метод цепочек


In [58]:
class HashTableChaining:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        if self.table[index] is None:
            self.table[index] = [(key, value)]
        else:
            for i, kv in enumerate(self.table[index]):
                k, v = kv
                if k == key:
                    self.table[index][i] = (key, value)
                    return
            self.table[index].append((key, value))

    def search(self, key):
        index = self.hash_function(key)
        if self.table[index] is None:
            return None
        for k, v in self.table[index]:
            if k == key:
                return v
        return None

    def delete(self, key):
        index = self.hash_function(key)
        if self.table[index] is None:
            return
        for i, kv in enumerate(self.table[index]):
            k, v = kv
            if k == key:
                del self.table[index][i]
                return


hash_table2 = HashTableChaining(100)
for num in first_date:
    hash_table2.insert(num, num)

time = timeit.timeit(lambda: hash_table2.search(166), number=1)
print(f"Время выполнения: {time} секунд")

result = hash_table2.search(166)
if result != -1:
    print(f"Число найдено: {result}")
else:
    print("Число не найдено")

Время выполнения: 4.900037311017513e-06 секунд
Число найдено: 166


### Задание №3


Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [60]:
def is_safe(board, row, col):
    for i in range(row):
        if board[i] == col or \
           board[i] - i == col - row or \
           board[i] + i == col + row:
            return False
    return True

def place_queens(board, row):
    if row == 8:
        return True
    for col in range(8):
        if is_safe(board, row, col):
            board[row] = col
            if place_queens(board, row + 1):
                return True
            board[row] = -1
    return False

def print_solution(board):
    for row in range(8):
        line = ['.'] * 8
        line[board[row]] = 'Q'
        print(' '.join(line))
    print()

def solve_n_queens():
    board = [-1] * 8
    if place_queens(board, 0):
        print_solution(board)
    else:
        print("Решений не найдено")

time = timeit.timeit(lambda:solve_n_queens(), number=1)
print(f"Время выполнения: {time} секунд")


Q . . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . Q . . . .

Время выполнения: 0.0015780999674461782 секунд
